In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
X = np.load('/kaggle/input/ptbxl-atrial-fibrillation-detection/ecgeq-500hzsrfava.npy')
Y = pd.read_csv('/kaggle/input/ptbxl-atrial-fibrillation-detection/coorteeqsrafva.csv',sep=';')

In [ ]:
sr = X[Y[Y['ritmi']=='SR'].index]
fa = X[Y[Y['ritmi']=='AF'].index]

In [ ]:
fa.shape

In [ ]:
Xeq = np.vstack([sr,fa])

In [ ]:
Xeq.shape

In [ ]:
labels = np.hstack((np.zeros(sr.shape[0]),np.ones(fa.shape[0])))

In [ ]:
type(labels)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils
from collections import Counter
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.layers import AveragePooling1D, Input

In [ ]:
from scipy.signal import butter, filtfilt
import numpy as np

def butter_bandpass(lcutoff,hcutoff, fs, order=5):
    nyq = 0.5 * fs
    lcoff = lcutoff / nyq
    hcoff = hcutoff / nyq
    b, a = butter(order, [lcoff,hcoff], btype='bandpass', analog=False)
    return b, a

def butter_bandpass_filter(data, lcutoff,hcutoff, fs, order=5):
    b, a = butter_bandpass(lcutoff,hcutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [ ]:
args = (10,200,5000,3)
fXeq = np.apply_along_axis(butter_bandpass_filter,1,Xeq,*args)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(fXeq[:,:,[1,6]],labels,test_size=.2,shuffle=True)

In [ ]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train.astype('float32')))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val.astype('float32')))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
Xeq = []
sr = []
fa = []
X = []

In [ ]:
#@tf.function
def resnet1d(data,nfilters,res):
    x = tf.expand_dims(data,-1)
    
    x = layers.Conv1D(256,100,strides=5,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv1D(256,50,strides=5,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    b1o = layers.BatchNormalization()(x)
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b1o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b2o = layers.add([x,b1o])

    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b2o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b3o = layers.add([x,b2o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b3o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b4o = layers.add([x,b3o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b4o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b5o = layers.add([x,b4o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b5o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b6o = layers.add([x,b5o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b6o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b7o = layers.add([x,b6o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b7o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b8o = layers.add([x,b7o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b8o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b9o = layers.add([x,b8o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b9o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b10o = layers.add([x,b9o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b10o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b11o = layers.add([x,b10o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b11o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b12o = layers.add([x,b11o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b12o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b13o = layers.add([x,b12o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b13o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b14o = layers.add([x,b13o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b14o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b15o = layers.add([x,b14o])
    
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(b15o)
    x = layers.Conv1D(nfilters,res,strides=1,activation='relu',padding='same')(x)
    x = layers.BatchNormalization()(x)
    b16o = layers.add([x,b15o])
    

    x = layers.Conv1D(nfilters*2,res,strides=1,activation='relu',padding='same')(b16o)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(.5)(x)
    x = layers.Dense(32)(x)
    x = layers.BatchNormalization()(x)

    return(x)

In [ ]:
#@tf.function
def logL1(ytrue,ypred):
    hazard_ratio = tf.exp(ypred)
    log_risk = tf.math.log(tf.reduce_sum(hazard_ratio))
    uncensored_likelihood = ypred - log_risk
    censored_likelihood = uncensored_likelihood * ytrue
    num_observed_events = tf.reduce_sum(ytrue)
    neg_likelihood = - tf.reduce_sum(censored_likelihood) / num_observed_events
    return neg_likelihood

def logL2(fail_indicator,logits):
    logL = 0
    #logits = 1e-5 * tf.reduce_sum(tf.square(logits))
    hazard_ratio = tf.math.exp(logits)
    cumsum_hazard_ratio = tf.cumsum(hazard_ratio,0)
    log_risk = tf.math.log(cumsum_hazard_ratio)
    likelihood = logits - log_risk
    uncensored_likelihood = likelihood * fail_indicator
    logL = - tf.reduce_sum(uncensored_likelihood,0)
    observation = tf.reduce_sum(fail_indicator,0) + 1
    return 1.0 * (logL / observation)# + 1e-3 * tf.reduce_sum(tf.square(logits))


In [ ]:
class ActivityRegularizationLayer(layers.Layer):
    def call(self, inputs):
        self.add_loss(1e-5 * tf.reduce_sum(tf.square(inputs)))
        return inputs


In [ ]:
# Instantiate an optimizer.
optimizer = keras.optimizers.SGD(learning_rate=1e-5,momentum=.99,nesterov=True)
#optimizer = keras.optimizers.Adam(learning_rate=1e-3)
# Instantiate a loss function.
#loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
loss_fn = logL2

train_acc_metric = keras.metrics.BinaryAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()

In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [ ]:
in1 = layers.Input(shape=(5000))
in2 = layers.Input(shape=(5000))
x1 = resnet1d(in1,16,10)
x2 = resnet1d(in2,16,10)
x = layers.add([x1,x2])
#x = ActivityRegularizationLayer()(x)
outputs = layers.Dense(1,'relu', kernel_regularizer=tf.keras.regularizers.L1(0.01), activity_regularizer=tf.keras.regularizers.L2(0.01))(x)

model = models.Model([in1,in2],outputs)
#model = CustomModel([in1,in2],outputs)

In [ ]:
model.compile(loss=loss_fn,optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.fit([x_train[:,:,0],x_train[:,:,1]],y_train,batch_size=12,epochs=20)

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
        # Add any extra losses created during the forward pass.
        loss_value += sum(model.losses)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

@tf.function
def test_step(x, y):
    val_logits = model(x, training=False)
    val_acc_metric.update_state(y, val_logits)

In [ ]:
import time

epochs = 10
for epoch in range(1,epochs+1):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        loss_value = train_step(x_batch_train, y_batch_train)
        #if np.isnan(loss_value.numpy()):
        #    break

        # Log every 200 batches.
        if step % 10 == 0:
            print("\rTraining loss at step %d: %.4f     " % (step, float(loss_value)),end='')
           # print("Seen so far: %d samples" % ((step + 1) * 12))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        test_step(x_batch_val, y_batch_val)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))

In [ ]:
# Test Model
#test_list = []
#for i in range(len(leads)):
#    test_list.append(Xtest[:,:,i])
yprob = model.predict([x_val[:,:,0],x_val[:,:,1]])

In [ ]:
yprob[:10]

In [ ]:
logL2(y_val.astype('float32'),yprob).shape

In [ ]:
cutoff =.5
yp = np.array([1 if x >= cutoff else 0 for x in yprob])
ya = y_val
cm = confusion_matrix(yp, ya)
print(cm)
print(classification_report(ya, yp, target_names=['nFA','FA']))